# Set up

Create a regression problem

In [30]:
import pandas as pd
import numpy as np

from sklearn.datasets import make_regression
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import r2_score

%matplotlib inline

In [2]:
X, y = make_regression(n_samples=1000)

r2_scores = dict()  # We will hold results here.

Use a Random Forest to model the problem

In [3]:
model = RandomForestRegressor(oob_score=True, n_estimators=100, n_jobs=-1)

In [4]:
model.fit(X, y)
r2_scores['Benchmark'] = model.oob_score_
pd.Series(r2_scores)

Benchmark    0.708331
dtype: float64

# Use the Transformer to calibrate

In [5]:
from sklearn.pipeline import Pipeline
from sklearn.base import TransformerMixin

from QuantileCalibrator import QuantileCalibrator

# Hacky way to change a RandomForest into a Transformer
class RandomForestTransformer(RandomForestRegressor, TransformerMixin):
    
    def transform(self, X, y=None):
        return self.predict(X)

In [6]:
rf = RandomForestTransformer(oob_score=True, n_estimators=100)
qc = QuantileCalibrator(quantile=100, isotonic_fit=True, isotonic_lambda=1)

steps = [
    ('random_forest', rf),
    ('quantile_cal', qc)
]

pipeline = Pipeline(steps=steps)

In [7]:
pipeline.fit(X, y)

Pipeline(memory=None,
     steps=[('random_forest', RandomForestTransformer(bootstrap=True, criterion='mse', max_depth=None,
            max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction...t=False)), ('quantile_cal', QuantileCalibrator(isotonic_fit=True, isotonic_lambda=1, quantile=100))])

Scoring like this will result in over fitting:

In [8]:
r2_scores['Pipeline Overfit'] = pipeline.score(X, y)
pd.Series(r2_scores)

Benchmark           0.708331
Pipeline Overfit    0.973773
dtype: float64

We can instead use the out-of-bag predictions:

In [9]:
r2_scores['Pipeline OOB'] = qc.score(rf.oob_prediction_, y)
pd.Series(r2_scores)

Benchmark           0.708331
Pipeline OOB        0.720122
Pipeline Overfit    0.973773
dtype: float64

# Cross Validate Results

Alternatively, we can use $k$-fold cross validation on the entire pipeline

In [10]:
from sklearn.model_selection import cross_validate, cross_val_score

In [11]:
cross_validated_scores = cross_val_score(X=X, y=y, cv=10, estimator=pipeline, n_jobs=-1)
r2_scores['Pipeline 10 Fold CV'] = cross_validated_scores.mean()
pd.Series(r2_scores)

Benchmark              0.708331
Pipeline 10 Fold CV    0.718306
Pipeline OOB           0.720122
Pipeline Overfit       0.973773
dtype: float64

# Hyper Parameter Search: 2 Steps

We can now optimize to find the best hyper parameters.

First, we'll do this with only the Random Forest.

In [12]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform

In [13]:
search_params = {
    'n_estimators': randint(10, 1000),
    'max_features': uniform(0, 1)
}

rf = RandomForestRegressor()

random_search = RandomizedSearchCV(estimator=rf, param_distributions=search_params, n_iter=30, n_jobs=-1, cv=10)

In [14]:
search_result = random_search.fit(X, y)
search_result.best_params_

{'max_features': 0.5670755973507031, 'n_estimators': 736}

In [15]:
# Train on the full dataset
rf.set_params(oob_score=True, **search_result.best_params_)
rf.fit(X, y)
r2_scores['RF Only, Best HPs'] = rf.oob_score_
pd.Series(r2_scores)

Benchmark              0.708331
Pipeline 10 Fold CV    0.718306
Pipeline OOB           0.720122
Pipeline Overfit       0.973773
RF Only, Best HPs      0.722126
dtype: float64

Next, we can fit a quantile calibrator using these parameters!

In [16]:
rf = RandomForestTransformer()
qc = QuantileCalibrator()

pipeline = Pipeline(steps=[('random_forest', rf), ('quantile_calibrator', qc)])

# We only need to fit params for the QuantileCalibrator because the RandomForest was already fit above.
search_params = {
    'random_forest__max_features': [search_result.best_params_['max_features']],
    'random_forest__n_estimators': [search_result.best_params_['n_estimators']],
    'quantile_calibrator__quantile': randint(10, 300),
    'quantile_calibrator__isotonic_fit': [True, False],
    'quantile_calibrator__isotonic_lambda': uniform(0.01, 20)
}

random_search = RandomizedSearchCV(estimator=pipeline, 
                                   param_distributions=search_params, 
                                   n_iter=30, 
                                   n_jobs=-1,
                                   verbose=1,
                                   cv=10)

In [17]:
search_result2 = random_search.fit(X, y)

Fitting 10 folds for each of 30 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Done 104 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  2.6min finished


In [18]:
search_result2.best_params_

{'quantile_calibrator__isotonic_fit': False,
 'quantile_calibrator__isotonic_lambda': 5.353728367828218,
 'quantile_calibrator__quantile': 14,
 'random_forest__max_features': 0.5670755973507031,
 'random_forest__n_estimators': 736}

In [19]:
# Train on the full dataset
pipeline.set_params(random_forest__oob_score=True, **search_result2.best_params_)
pipeline.fit(X, y)
rf_pred = pipeline.named_steps['random_forest'].oob_prediction_
r2_scores['Pipeline Best HP 2 steps'] = pipeline.named_steps['quantile_calibrator'].score(rf_pred, y)
pd.Series(r2_scores)

Benchmark                   0.708331
Pipeline 10 Fold CV         0.718306
Pipeline Best HP 2 steps    0.753585
Pipeline OOB                0.720122
Pipeline Overfit            0.973773
RF Only, Best HPs           0.722126
dtype: float64

# Hyper Parameter Search: 1 Step

We can also search for the best HPs for both stages of the pipeline simultaniously.

In [20]:
rf = RandomForestTransformer()
qc = QuantileCalibrator()

pipeline = Pipeline(steps=[('random_forest', rf), ('quantile_calibrator', qc)])

# We only need to fit params for the QuantileCalibrator because the RandomForest was already fit above.
search_params = {
    'random_forest__max_features': uniform(0.1, 0.9),
    'random_forest__n_estimators': randint(10, 1000),
    'random_forest__n_jobs': [-1],
    'quantile_calibrator__quantile': randint(10, 300),
    'quantile_calibrator__isotonic_fit': [True, False],
    'quantile_calibrator__isotonic_lambda': uniform(0.01, 20)
}

random_search = RandomizedSearchCV(estimator=pipeline, 
                                   param_distributions=search_params, 
                                   n_iter=30, 
                                   n_jobs=-1,
                                   verbose=1,
                                   cv=10)

In [21]:
search_result3 = random_search.fit(X, y)
search_result3.best_params_

Fitting 10 folds for each of 30 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Done 104 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  2.1min finished


{'quantile_calibrator__isotonic_fit': False,
 'quantile_calibrator__isotonic_lambda': 0.0855219764353443,
 'quantile_calibrator__quantile': 280,
 'random_forest__max_features': 0.3932599483489593,
 'random_forest__n_estimators': 502,
 'random_forest__n_jobs': -1}

In [22]:
r2_scores['Pipeline Best HP 1 step'] = search_result3.best_score_.mean()
pd.Series(r2_scores)

Benchmark                   0.708331
Pipeline 10 Fold CV         0.718306
Pipeline Best HP 1 step     0.749277
Pipeline Best HP 2 steps    0.753585
Pipeline OOB                0.720122
Pipeline Overfit            0.973773
RF Only, Best HPs           0.722126
dtype: float64

## Conclusion

In this example, it appears that the best results are found by optimizing one stage of the pipeline at a time (ignoring the overfit result).